In [6]:
import numpy as np
import cv2
import depthai
import time

In [7]:
# Closer-in minimum depth, disparity range is doubled (from 95 to 190):
extended_disparity = False
# Better accuracy for longer distance, fractional disparity 32-levels:
subpixel = False
# Better handling for occlusions:
lr_check = True

#create pipeline for the RGB color camera
pipeline = depthai.Pipeline()
rgb = pipeline.create(depthai.node.ColorCamera)
rgb.setPreviewSize(640,480)
rgb.setInterleaved(False)

#create pipeline for the right mono camera
right = pipeline.create(depthai.node.MonoCamera)
right.setBoardSocket(depthai.CameraBoardSocket.RIGHT)
right.setResolution(depthai.MonoCameraProperties.SensorResolution.THE_480_P)

#create pipeline for the left mono camera
left = pipeline.create(depthai.node.MonoCamera)
left.setBoardSocket(depthai.CameraBoardSocket.LEFT)
left.setResolution(depthai.MonoCameraProperties.SensorResolution.THE_480_P)

#create pipeline for the stereo depth camera using the left and right cameras
depth = pipeline.create(depthai.node.StereoDepth)
xout = pipeline.create(depthai.node.XLinkOut)
xout_rgb = pipeline.create(depthai.node.XLinkOut)

#set the stream names for both RGB and stereo depth cameras in order for them to be used in the pipeline for display
xout_rgb.setStreamName("rgb")
xout.setStreamName("disparity")

# Create a node that will produce the depth map (using disparity output as it's easier to visualize depth this way)
depth.setDefaultProfilePreset(depthai.node.StereoDepth.PresetMode.HIGH_DENSITY)
# Options: MEDIAN_OFF, KERNEL_3x3, KERNEL_5x5, KERNEL_7x7 (default)
depth.initialConfig.setMedianFilter(depthai.MedianFilter.KERNEL_7x7)
depth.setLeftRightCheck(lr_check)
depth.setExtendedDisparity(extended_disparity)
depth.setSubpixel(subpixel)

# Linking all of the camera nodes together for use in the pipeline
rgb.preview.link(xout_rgb.input)
left.out.link(depth.left)
right.out.link(depth.right)
depth.disparity.link(xout.input)

In [8]:
#gather the frame rates seen by the camera while running both
prev_frame_time = 0
new_frame_time = 0

with depthai.Device(pipeline) as device:
    # Output queue will be used to get the depth and rgb frames from the outputs defined above
    q_depth = device.getOutputQueue(name="disparity", maxSize=1, blocking=False)
    q_rgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    #create a list to store the fps captures as they happen
    fps_captures = list()
    while True:
        #get the stereo cameras frames
        inDisparity = q_depth.get()
        frame = inDisparity.getFrame()
        # Normalization for better visualization
        frame = (frame * (255 / depth.initialConfig.getMaxDisparity())).astype(np.uint8)
        frame = cv2.applyColorMap(frame, cv2.COLORMAP_JET)
        cv2.imshow("disparity_color", frame)
        #get the rgb camera frames
        in_rgb = q_rgb.tryGet()

        #add the fps to the RGB camera so it can be seen as the frames change per second
        if in_rgb is not None:
            frame = in_rgb.getCvFrame()
            new_frame_time = time.time()
            fps = 1/(new_frame_time-prev_frame_time)
            fps_captures.append(fps)
            prev_frame_time = new_frame_time
            fps = str(int(fps))
            cv2.putText(frame, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)
            cv2.imshow("rgb", frame)
        #exit code to kill the program
        if cv2.waitKey(1) == ord('q'):
            break
cv2.destroyAllWindows()

In [9]:
#display the average and the median fps experienced by the camera during the run
np_fps = np.array(fps_captures)
np_fps.sort()
print("avg fps: ",int(np_fps.mean()))
print("median fps: ",int(np_fps[int(len(np_fps)/2)]))

avg fps:  33
median fps:  27
